In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from zipfile import ZipFile

with ZipFile("data (1).zip", 'r') as zObject:
    zObject.extractall(path="")

In [3]:
SOS_token = 0
EOS_token = 1

class Lang:
  def __init__(self, name):
    self.name = name
    self.word2index = {}
    self.word2count = {}
    self.index2word = {0:"SOS", 1:"EOS"}
    self.n_words = 2
  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)
  def addWord(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.n_words
      self.word2count[word] = 1
      self.index2word[self.n_words] = word
      self.n_words += 1
    else:
      self.word2count[word] += 1

In [4]:
def unicodeToAscii(s):
  return ''.join(
      c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn'
  )

def normalizeString(s):
  s = unicodeToAscii(s.lower().strip())
  s = re.sub(r"([.!?])", r" \1", s)
  s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
  return s.strip()

In [5]:
def readLangs(lang1, lang2, reverse=False):
  print("Reading Lines......")

  lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')

  pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

  if reverse:
    pairs = [list(reversed(p)) for p in pairs]
    input_lang = Lang(lang2)
    output_lang = Lang(lang1)
  else:
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

  return input_lang, output_lang, pairs

In [6]:
MAX_LEN = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
  return len(p[0].split(' ')) < MAX_LEN and len(p[1].split(' ')) < MAX_LEN and p[1].startswith(eng_prefixes)

def filterPairs(pairs):
  return [p for p in pairs if filterPair(p)]

In [7]:
def prepareData(lang1, lang2, reverse = False):
  input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
  print("Read %s sentence pairs" %len(pairs))
  pairs = filterPairs(pairs)
  print("Trimmed to %s sentence pairs" % len(pairs))
  print("Counting words....")
  for pair in pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])
  print("Counted Words")
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name, output_lang.n_words)
  return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng','fra',True)

Reading Lines......
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words....
Counted Words
fra 4601
eng 2991


In [8]:
print(random.choice(pairs))

['je ne suis pas un bon menteur', 'i m not a good liar']


In [9]:
class EncoderRNN(nn.Module):
  def __init__(self, input_size, hidden_size, dropout_p=0.1):
    super(EncoderRNN, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
    self.dropout = nn.Dropout(dropout_p)

  def forward(self, input):
    embedded = self.dropout(self.embedding(input))
    # print('embedded :', embedded.shape)
    output, hidden = self.gru(embedded)
    # print('output :', output.shape)
    # print('Hidden :', hidden[0].shape)
    return output, hidden

In [21]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LEN):
            decoder_output, decoder_hidden = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                decoder_input = target_tensor[:, i].unsqueeze(1)
            else:
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()
        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [11]:
def indexesFromSentence(lang, sentence):
  return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
  indexes = indexesFromSentence(lang, sentence)
  indexes.append(EOS_token)
  return torch.tensor(indexes, dtype=torch.long, device=device).view(1,-1)

def tensorsFromPair(pair):
  input_tensor = tensorFromSentence(input_lang, pair[0])
  target_tensor = tensorFromSentence(output_lang, pair[1])
  return (input_tensor, target_tensor)

def get_dataloader(batch_size):
  input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

  n = len(pairs)
  input_ids = np.zeros((n,MAX_LEN), dtype=np.int32)
  target_ids = np.zeros((n,MAX_LEN), dtype=np.int32)

  for idx, (inp, tgt) in enumerate(pairs):
    inp_ids = indexesFromSentence(input_lang, inp)
    tgt_ids = indexesFromSentence(output_lang, tgt)
    inp_ids.append(EOS_token)
    tgt_ids.append(EOS_token)
    input_ids[idx, :len(inp_ids)] = inp_ids
    target_ids[idx, :len(tgt_ids)] = tgt_ids

  train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                             torch.LongTensor(target_ids).to(device))

  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
  return input_lang, output_lang, train_dataloader



In [12]:
def train_epoch(data_loader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):

  total_loss = 0
  for data in data_loader:
    input_tensor, target_tensor = data
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    encoder_outputs, encoder_hidden = encoder(input_tensor)
    decoder_outputs,_,_ = decoder(encoder_outputs, encoder_hidden, target_tensor)
    loss = criterion(
        decoder_outputs.view(-1, decoder_outputs.size(-1)),
        target_tensor.view(-1)
    )
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    total_loss += loss.item()
  return total_loss / len(data_loader)


In [13]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [14]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001, print_every=100, plot_every=100):
  start = time.time()
  plot_losses = []
  print_loss_total = 0
  plot_loss_total = 0
  encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
  decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
  criterion = nn.NLLLoss()

  for epoch in range(1, n_epochs + 1):
    loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
    print_loss_total += loss
    if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))


In [24]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
  with torch.no_grad():
    input_tensor = tensorFromSentence(input_lang, sentence)

    encoder_outputs, encoder_hidden = encoder(input_tensor)

    decoder_outputs, decoder_hidden, decoder_attention = decoder(encoder_outputs, encoder_hidden)

    _, topi = decoder_outputs.topk(1)
    decoded_ids = topi.squeeze()
    decoded_words = []

    for idx in decoded_ids:
      if idx.item()==EOS_token:
        decoded_words.append('<EOS>')
        break
      else:
        decoded_words.append(output_lang.index2word[idx.item()])
  return decoded_words, decoder_attention

In [16]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [22]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

Reading Lines......
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words....
Counted Words
fra 4601
eng 2991
0m 32s (- 8m 2s) (5 6%) 1.6980
1m 2s (- 7m 19s) (10 12%) 0.9260
1m 31s (- 6m 37s) (15 18%) 0.6359
2m 0s (- 6m 1s) (20 25%) 0.4639
2m 32s (- 5m 34s) (25 31%) 0.3488
3m 7s (- 5m 13s) (30 37%) 0.2698
3m 42s (- 4m 45s) (35 43%) 0.2108
4m 12s (- 4m 12s) (40 50%) 0.1690
4m 44s (- 3m 41s) (45 56%) 0.1366
5m 16s (- 3m 9s) (50 62%) 0.1136
5m 47s (- 2m 37s) (55 68%) 0.0953
6m 19s (- 2m 6s) (60 75%) 0.0828
6m 51s (- 1m 34s) (65 81%) 0.0721
7m 22s (- 1m 3s) (70 87%) 0.0644
7m 55s (- 0m 31s) (75 93%) 0.0582
8m 26s (- 0m 0s) (80 100%) 0.0543


In [25]:
evaluateRandomly(encoder, decoder, 20)

> vous etes sur mon chemin
= you are in my way
< you are good at home <EOS>

> nous sommes timides
= we re shy
< we re happy <EOS>

> je prends mon apres midi demain
= i m taking tomorrow afternoon off
< i m taking tomorrow afternoon off <EOS>

> je m inquiete pour l avenir
= i m worried about the future
< i m just trying for the office room <EOS>

> tu n es pas le seul avec ce probleme
= you re not the only one with this problem
< you re not the only one with this problem <EOS>

> il patine
= he is skating
< he is skating <EOS>

> vous etes fort attirant
= you re very attractive
< you re very attractive in blue <EOS>

> je suis tres sensible au froid
= i am very sensitive to the cold
< i am very sensitive to the cold weather <EOS>

> elle a de l assurance
= she s assertive
< she s new here aren t you ? <EOS>

> je suis impatient de vous rencontrer
= i am looking forward to seeing you
< i am looking forward to seeing you again <EOS>

> c est un garcon intelligent
= he is a clever boy
<